# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tual,-5.6667,132.7500,25.57,77,31,9.94,ID,1691455533
1,1,ushuaia,-54.8000,-68.3000,2.81,75,0,1.03,AR,1691455533
2,2,adamstown,-25.0660,-130.1015,20.88,76,72,3.54,PN,1691455533
3,3,sarina,-21.4333,149.2167,24.04,61,1,5.44,AU,1691455533
4,4,hadibu,12.6500,54.0333,25.58,80,41,9.97,YE,1691455534


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
len(city_data_df)

580

In [35]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng", "Lat",geo = True,tiles = "OSM",frame_width = 800,frame_height = 600,size = "Humidity",color="City",alpha=.75)

# Display the map
map_plot_1
#hvplot.show(map_plot_1)    # if you have to view in locoalhost

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
#wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
# cloudiness zero
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
190,190,nova vicosa,-17.8919,-39.3719,22.56,82,0,4.25,BR,1691455577
201,201,international falls,48.6011,-93.4110,24.10,46,0,3.60,US,1691455581
238,238,ribas do rio pardo,-20.4431,-53.7592,24.90,36,0,0.66,BR,1691455188
306,306,paveh,35.0434,46.3565,23.67,31,0,2.96,IR,1691455605
313,313,balykshi,47.0667,51.8667,22.00,46,0,2.00,KZ,1691455399
329,329,nouadhibou,20.9310,-17.0347,21.99,83,0,4.12,MR,1691455611
364,364,griffith,41.5284,-87.4237,23.96,73,0,3.60,US,1691455619
386,386,lata,40.1629,-8.3327,21.71,32,0,1.72,PT,1691455394
387,387,odessa,46.4775,30.7326,22.17,54,0,2.84,UA,1691455622
411,411,khadan khak,30.7477,67.7085,22.09,35,0,1.89,PK,1691455630


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Create a copy of the original DataFrame with selected columns
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
190,nova vicosa,BR,-17.8919,-39.3719,82,
201,international falls,US,48.6011,-93.4110,46,
238,ribas do rio pardo,BR,-20.4431,-53.7592,36,
306,paveh,IR,35.0434,46.3565,31,
313,balykshi,KZ,47.0667,51.8667,46,
329,nouadhibou,MR,20.9310,-17.0347,83,
364,griffith,US,41.5284,-87.4237,73,
386,lata,PT,40.1629,-8.3327,32,
387,odessa,UA,46.4775,30.7326,54,
411,khadan khak,PK,30.7477,67.7085,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters (10 km) radius
# params is a dictionary used to define the parameters for the Geoapify API request.
params = {
    "apiKey": geoapify_key,  # Replace with your actual Geoapify API key
    "categories": "accommodation.hotel", #Visit the official Geoapify API documentation website,To find category IDs for the Geoapify API.
    "radius" : 10000
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    # explored https://apidocs.geoapify.com/docs/places/#categories to find information
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params)
    #The response from the API is stored in the variable name_address.
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nova vicosa - nearest hotel: Pousada Pontal da Barra
international falls - nearest hotel: Copper River Inn & Conference Centre
ribas do rio pardo - nearest hotel: Hotel São Francisco
paveh - nearest hotel: هتل ارم
balykshi - nearest hotel: Арасан
nouadhibou - nearest hotel: Hotel valencia
griffith - nearest hotel: Quality Inn & Suites
lata - nearest hotel: Residencial Botânico
odessa - nearest hotel: Готель Венера
khadan khak - nearest hotel: No hotel found
alpena - nearest hotel: Ramada of Alpena
afif - nearest hotel: Samaya Royal Afif
beyneu - nearest hotel: Johan yu
jaragua - nearest hotel: Boa Vista Park Hotel
pontes e lacerda - nearest hotel: Hotel Avenida
al jawf - nearest hotel: No hotel found
robore - nearest hotel: Lajas
palmas - nearest hotel: Hotel Triângulo Mineiro
tongliao - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
190,nova vicosa,BR,-17.8919,-39.3719,82,Pousada Pontal da Barra
201,international falls,US,48.6011,-93.4110,46,Copper River Inn & Conference Centre
238,ribas do rio pardo,BR,-20.4431,-53.7592,36,Hotel São Francisco
306,paveh,IR,35.0434,46.3565,31,هتل ارم
313,balykshi,KZ,47.0667,51.8667,46,Арасан
329,nouadhibou,MR,20.9310,-17.0347,83,Hotel valencia
364,griffith,US,41.5284,-87.4237,73,Quality Inn & Suites
386,lata,PT,40.1629,-8.3327,32,Residencial Botânico
387,odessa,UA,46.4775,30.7326,54,Готель Венера
411,khadan khak,PK,30.7477,67.7085,35,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points( "Lng","Lat",geo = True,tiles = "OSM", frame_width = 800,frame_height = 600,size = "Humidity",color="City",alpha=.75,hover_cols=["Hotel Name", "Country"])

# Display the map
map_plot_2



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)